Installing dependencies and packages

In [ ]:
import os
os.chdir('/content/drive/MyDrive/projects_assignment/aadhar_card_retrival')
os.getcwd()

'/content/drive/MyDrive/projects_assignment/aadhar_card_retrival'

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (8,316 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
!pip install Image

In [ ]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [ ]:
import pytesseract
from PIL import Image
import urllib.request
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
import spacy

### Data Preparation for the Annotation
We'll take each Aadhar image pass it through the Tesseract and get the text from image.<br>
We'll put all text in a .txt file as our annotator accepts this format only. Each line in .txt file represents one Aadhar image<br>
<b>You can explore annotation software from here</b><br>
https://github.com/rumankhan1/ner-annotator

In [ ]:
def get_text_data(path_to_img):

  '''
  This method takes image file path then given to the tesseract to apply ocr and get the text(string)
  '''

  try:
    #now we'll get the text from image through tesserac
    return pytesseract.image_to_string(Image.open(path_to_img))
  except:
    return ' '


In [ ]:
print(get_text_data('aadhar_images_to_train_model/59b6f819e3c398379524acd67662618e.jpg'))

In [ ]:
os.listdir()

['info.ipynb', 'aadhar_images_to_train_model']

In [ ]:
#first get all the images present in the directory
all_aadhar_files = os.listdir('aadhar_images_to_train_model')

# This code will create a .txt file and write every string from the Aadhar image into new line
with open('aadhar_train_50.txt','w') as out:
  one_line_str = ''
  for file_path in tqdm(all_aadhar_files):
    #looping through each aadhar image path and getting text out of that
    file_path = 'aadhar_images_to_train_model/'+file_path
    temp_str = ' '.join(get_text_data(file_path).split('\n'))
    one_line_str+= temp_str + '\n'
  out.write(one_line_str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


### Performing NER Annotation
Head to this https://github.com/rumankhan1/ner-annotator software and perform the annotation <br>
At the end we'll have training_data.json for training the spacy

In [ ]:
os.listdir()

['info.ipynb',
 'aadhar_images_to_train_model',
 'aadhar_train_50.txt',
 'training_data.json']

### Training Spacy on custom data
In involves multiple step, Let's walk throught that

In [ ]:
#importing relevant packages 
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
import json

#### Loading the cutom training data

In [ ]:
os.listdir()

['info.ipynb',
 'aadhar_images_to_train_model',
 'aadhar_train_50.txt',
 'training_data.json']

In [ ]:
with open('training_data.json') as data_file:
  data = data_file.read()
  print(data) 
  data_content = json.loads(data)

{"classes":["DOB","NAME","FATHER_NAME","AADHAR_NUMBER","GENDER","ADDRESS","MOBILE_NUMBER","AADHAR_VID"],"annotations":[["Shubham ‘ fefe/DOB : 05/03/1997 yew MALE Mobile No : 8800780509 4275 2080 8052 2a aren , 2X) vears",{"entities":[[0,7,"NAME"],[20,30,"DOB"],[35,39,"GENDER"],[52,62,"MOBILE_NUMBER"],[67,81,"AADHAR_NUMBER"]]}],["mY are om: S10 2. ee, sp gx, arf, SUR Wee - 221208 Address: S/O B.K. Sharma, Narayan Pur, Varanasi, Uttar Pradesh - 22120! 7 2998 5205 ViD : 9190 2926 6600 1448.",{"entities":[[82,144,"ADDRESS"]]}],["S Father ABDUL FATEH S54 AT Year of Birth : 1994 51/Femate 6216 2176 3960 SITS ~ SIRT STC orice",{"entities":[[6,24,"FATHER_NAME"],[49,53,"DOB"],[69,83,"AADHAR_NUMBER"]]}],["geveujun Car Aishwarya G mS Men / DOB : 17/11/1999 Quetorumed / Female AMG - ETSMYOO WeAGelor gH\\emIyj wo",{"entities":[[26,41,"NAME"],[54,64,"DOB"],[78,84,"GENDER"]]}],["Government of India Jaa aa ij Jain DOB : 02/01/1978",{"entities":[]}],["8689 2565 8268 | - 30a ates ar aire _ er) Male 8689

In [ ]:
# store all of annotated sample in a list which will be later used to train the Spacy

print('total annotated smaple for the training : ', len(data_content.get('annotations')))
training_data = data_content.get('annotations')

total annotated smaple for the training :  38


All the available entities

In [ ]:
data_content.get('classes')

['DOB',
 'NAME',
 'FATHER_NAME',
 'AADHAR_NUMBER',
 'GENDER',
 'ADDRESS',
 'MOBILE_NUMBER',
 'AADHAR_VID']

Defining the variables required to train the model. <br>
You can experiment with different Iterations ,dropout, etc

In [ ]:
model = None
output_dir = Path(os.getcwd())
n_iter=250

To train a custom Spacy we'll have to do two things first:


1.   Initialize a blank model
2.   Setup pipeline with only NER usig create_pipe function



In [ ]:
#load the model
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [ ]:
# let's verify the models in pipeline
nlp.pipe_names

['ner']

Adding labels to the model for the named entity. We'll have to get all the Named Entity from annotations.

In [ ]:
for _, annotations in training_data:

  #this loops through each text entities
    for ent in annotations.get('entities'):

      #adds those entities to the ner model which we're going to train
      ner.add_label(ent[2]) 

<b>Note: </b> One more thing, we'll have to make sure that we're only training for the NER and others are disbaled in the pipleline. 

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
other_pipes # no others recognizer in pipeline

[]

Now, It's time to train the model finally. <br>
One thing to note here, I'm just updating the spacy model with our own cutom dataset. To trian a custom model from scratch it requires huge data. 

In [ ]:
with nlp.disable_pipes(*other_pipes):  # only train NER, disables others in pipeline
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(training_data)
        losses = {}
        for text, annotations in tqdm(training_data):
            nlp.update([text], [annotations],drop=0.5,sgd=optimizer, losses=losses)
        print(losses)

  5%|▌         | 2/38 [00:00<00:02, 15.30it/s]

{'ner': 263.47309134765527}


  8%|▊         | 3/38 [00:00<00:01, 21.40it/s]

{'ner': 23.98043937964628}


  5%|▌         | 2/38 [00:00<00:02, 13.35it/s]

...

{'ner': 4.294484270401535}


 11%|█         | 4/38 [00:00<00:01, 32.05it/s]

{'ner': 2.6675489605762603}


100%|██████████| 38/38 [00:01<00:00, 21.23it/s]

{'ner': 11.880593357779382}


Let's save the model on local disk

In [ ]:
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/drive/My Drive/projects_assignment/aadhar_card_retrival


#### Testing the trained model on our own custom dataset <br>
<b>Note : </b> These all the text are ouput from tesserect which I'm passing as an argument to the show_entities() to detect the entities.

In [ ]:
def show_entities(text):
  print(text)
  print('\n')
  doc = nlp(text)
  for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
show_entities(training_data[28][0])

Devendra Singh ia fafa DOB : 01/01/1984 veo /MALE 7996 3003 9447 ao ware, TA gear mc


Devendra Singh NAME
7996 3003 9447 AADHAR_NUMBER


In [ ]:
show_entities(training_data[8][0])

cation Au ly of India ~ Address : 3-106 A, HEJAMADI, UDUPI, Hejamadi, Hejmadi, Udupi, Karnataka , 574103 2.




In [ ]:
show_entities(training_data[21][0])

avfaeaz ite 44 HARVINDER SINGH SANDHU) far: aaare fate 4 Father: SATNAM SINGH SANDHU aH a / Year of Birth : 1966 [eT / Male


eT / DOB


In [ ]:
show_entities(training_data[32][0])

aa / Name: Sid malhorta Harta / DOB : 28/05/2000 yea / Female


Sid malhorta DOB
28/05/2000 yea DOB


### Conclusion

We could improve this model if trained on larger dataset